# Class 7 - Performance



In [7]:
x = 5.0
y = 4
z = x + 1im * y
@show typeof(z)

typeof(z) = Complex{Float64}

Complex{Float64}

In [8]:
x = 5
y = 4
z1 = x + 1im * y
z2 = Complex{Float64}(x,y)
@show typeof(z1)
@show typeof(z2)
;

typeof(z1) = Complex{Int64}
typeof(z2) = Complex{Float64}


In [11]:
z = Complex128(1,1)
@show typeof(z)

typeof(z) = Complex{Float64}

Complex{Float64}

In [16]:
x = BigFloat(1) + Bi

1.100000000000000000000000000000000000000000000000000000000000000000000000000000e+01